# Evaluación Predictiva antes y después de la Limpieza de Datos

Este notebook tiene como objetivo comparar el desempeño de un modelo predictivo utilizando un conjunto de datos **antes y después** de aplicar una limpieza adecuada.

Analizaremos el impacto de la imputación, codificación y escalado de variables, mostrando cómo cada decisión afecta la calidad del modelo.

La variable objetivo será `Alta_conectividad`, una clasificación binaria basada en el tiempo de uso de internet por día.


# 🧩 Estudio de caso: Conectividad y bienestar social

## 📘 Contexto general

La Fundación Datos para el Progreso Social está trabajando en un programa que busca **mejorar el acceso a internet en comunidades vulnerables** de zonas urbanas y rurales. Para justificar futuras inversiones, el equipo de análisis ha recolectado datos de **1.300 personas** a través de encuestas presenciales y registros administrativos.

El objetivo es entender cómo la **edad**, el **ingreso mensual**, el **nivel educativo** y el **uso diario de internet** se relacionan con la **posibilidad de acceder a oportunidades de formación, empleo remoto o trámites digitales**.

---

## 📊 Descripción del dataset

El archivo `dataset_ejemplo_1300.csv` contiene las siguientes variables:

| Variable            | Tipo          | Descripción |
|---------------------|---------------|-------------|
| `ID`                | Entero        | Identificador único por persona |
| `Edad`              | Numérica      | Edad en años |
| `Ingreso`           | Numérica      | Ingreso mensual en pesos |
| `Nivel_Educativo`   | Ordinal       | Básico, Medio, Superior |
| `Genero`            | Categórica    | Hombre / Mujer |
| `Horas_Internet`    | Numérica      | Promedio de horas de uso diario de internet |
| `Ciudad`            | Categórica    | Nombre de la ciudad de residencia |

---

## 🎯 Problema a resolver

Antes de construir un modelo predictivo que clasifique si una persona tiene alta conectividad (por ejemplo, más de 3.5 horas al día en internet), **es fundamental preparar los datos adecuadamente**.

El dataset presenta varios desafíos:
- **Valores faltantes** en ingresos y edad.
- **Codificación no numérica** en género y nivel educativo.
- **Posibles duplicados o inconsistencias** por errores en captura manual.

---

## 🔧 ¿Qué haremos en esta sesión?

Aplicaremos un flujo de trabajo de limpieza de datos inspirado en la metodología **CRISP-DM**, específicamente en la etapa de *Preparación de los Datos*. Vamos a:

1. Diagnosticar problemas comunes en el dataset.
2. Imputar valores faltantes.
3. Codificar variables categóricas.
4. Eliminar duplicados.
5. Generar una versión limpia y lista para modelar.


## 1. Cargar el dataset y crear variable objetivo

In [1]:
import pandas as pd

# Cargar el dataset
df_raw = pd.read_csv('../../datos/dataset_ejemplo_1300.csv')

# Crear variable objetivo: Alta conectividad
df_raw["Alta_conectividad"] = (df_raw["Horas_Internet"] > 3.5).astype(int)

df_raw.head()

,ID,Edad,Genero,Ingreso,Nivel_Educativo,Horas_Internet,Ciudad,CodigoID,Alta_conectividad
0,1,22.0,Femenino,4842.42,Secundaria,1.6,Cali,1-Cal,0
1,2,47.0,Masculino,5742.02,Posgrado,5.6,Medellín,2-Med,1
2,3,38.0,Otro,4335.12,Técnico,2.3,Villavicencio,3-Vil,0
3,4,17.0,Otro,4515.57,Tecnólogo,3.2,Bogotá,4-Bog,0
4,5,28.0,Otro,3846.49,Primaria,1.3,Pasto,5-Pas,0


### 📘 Explicación del código

- `import pandas as pd`:  
  Importa la librería `pandas`, usada para manipular datos en formato tabular.

- `pd.read_csv(...)`:  
  Carga el archivo CSV y lo convierte en un DataFrame llamado `df_raw`, que es una tabla de datos que permite filtrado, estadísticas, etc.

- `df_raw["Alta_conectividad"] = ...`:  
  Crea una nueva columna llamada `Alta_conectividad`, que será nuestra **variable objetivo**:
  - Asigna el valor `1` si la persona usa más de 3.5 horas diarias de internet.
  - Asigna el valor `0` en caso contrario.
  - Se convierte el resultado booleano en entero con `.astype(int)`.

- `df_raw.head()`:  
  Muestra las primeras 5 filas del DataFrame, útil para verificar que la carga de datos fue exitosa y que la nueva columna fue creada correctamente.


## 2. Evaluación del modelo antes de limpiar los datos

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Usamos solo Edad e Ingreso, eliminando nulos
df_before = df_raw[["Edad", "Ingreso"]].dropna()
Xb = df_before
yb = df_raw.loc[df_before.index, "Alta_conectividad"]

Xb_train, Xb_test, yb_train, yb_test = train_test_split(Xb, yb, test_size=0.3, random_state=42)

model_before = LogisticRegression(max_iter=200)
model_before.fit(Xb_train, yb_train)
yb_pred = model_before.predict(Xb_test)
acc_before = accuracy_score(yb_test, yb_pred)

print("✅ Exactitud antes de la limpieza:", round(acc_before, 4))

✅ Exactitud antes de la limpieza: 0.5778


### Explicación detallada del bloque de código: modelo antes de la limpieza

Este bloque evalúa el rendimiento de un modelo de regresión logística antes de realizar cualquier tipo de limpieza, escalado o codificación de datos. Se trabaja únicamente con las columnas `Edad` e `Ingreso`, eliminando registros con datos faltantes. El objetivo es establecer una línea base para comparar con el modelo que se construirá después de preparar los datos correctamente.

```python
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Usamos solo Edad e Ingreso, eliminando nulos
df_before = df_raw[["Edad", "Ingreso"]].dropna()
Xb = df_before
yb = df_raw.loc[df_before.index, "Alta_conectividad"]

Xb_train, Xb_test, yb_train, yb_test = train_test_split(Xb, yb, test_size=0.3, random_state=42)

model_before = LogisticRegression(max_iter=200)
model_before.fit(Xb_train, yb_train)
yb_pred = model_before.predict(Xb_test)
acc_before = accuracy_score(yb_test, yb_pred)

print("✅ Exactitud antes de la limpieza:", round(acc_before, 4))


### ¿Qué significa este resultado?

`Exactitud antes de la limpieza: 0.5778`

Este valor indica que el modelo de regresión logística logró una **exactitud del 57.78%** al predecir si una persona tiene o no alta conectividad, utilizando únicamente las variables `Edad` e `Ingreso`, y **sin aplicar limpieza, imputación o transformación de los datos**.

---

### ¿Qué es la exactitud?

La **exactitud (accuracy)** es una métrica de evaluación que indica qué proporción de predicciones del modelo fueron correctas. Se calcula así:

**Exactitud = predicciones correctas / total de predicciones**

En este caso, el modelo acertó en aproximadamente **58 de cada 100 predicciones**.

---

### ¿Cómo interpretar este valor?

- El modelo tiene un rendimiento **básico o limitado**.
- Aunque supera el azar (50% si las clases están balanceadas), aún está lejos de ser un modelo confiable.
- Esto sugiere que el modelo **no está capturando patrones relevantes en los datos**.

---

### ¿Por qué la exactitud es baja?

- Solo se usaron dos variables (`Edad` e `Ingreso`).
- Se eliminaron filas con valores faltantes (`dropna()`), lo que **reduce el tamaño de la muestra**.
- No se aplicó ningún tipo de **escalado**, **codificación de variables categóricas**, ni **imputación de valores**.

---

### ¿Para qué sirve este resultado?

Este resultado actúa como una **línea base**.  
Servirá como referencia para comparar con un segundo modelo entrenado después de aplicar técnicas de preparación de datos.  
Si ese segundo modelo obtiene una mayor exactitud, podremos concluir que **la preparación de datos mejora la capacidad predictiva del modelo**.


## 3. Preparación del dataset: limpieza, imputación y codificación

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

df_clean = df_raw.copy()

# Imputación
imputer_edad = SimpleImputer(strategy='median')
df_clean['Edad'] = imputer_edad.fit_transform(df_clean[['Edad']])

imputer_ingreso = SimpleImputer(strategy='mean')
df_clean['Ingreso'] = imputer_ingreso.fit_transform(df_clean[['Ingreso']])

# Codificación de variables categóricas
df_clean = pd.get_dummies(df_clean, columns=['Genero', 'Nivel_Educativo', 'Ciudad'], drop_first=True)

# Escalado
scaler = MinMaxScaler()
df_clean[['Edad', 'Ingreso']] = scaler.fit_transform(df_clean[['Edad', 'Ingreso']])

### 🧼 Limpieza y preparación del dataset

- `from sklearn.impute import SimpleImputer`:  
  Importa la clase `SimpleImputer` para rellenar valores faltantes en columnas numéricas.

- `from sklearn.preprocessing import MinMaxScaler`:  
  Importa el escalador `MinMaxScaler` para normalizar valores entre 0 y 1.

- `df_clean = df_raw.copy()`:  
  Crea una copia del DataFrame original para no modificar los datos en bruto.

---

#### 🔧 Imputación de valores faltantes

- `SimpleImputer(strategy='median')`:  
  Crea un imputador que reemplaza los valores nulos en la variable `Edad` por la mediana de la columna.

- `SimpleImputer(strategy='mean')`:  
  Reemplaza los valores nulos en la variable `Ingreso` por la media de la columna.

> Se utiliza `.fit_transform(...)` para ajustar el imputador y aplicar la transformación directamente.

---

#### 🏷️ Codificación de variables categóricas

- `pd.get_dummies(..., drop_first=True)`:  
  Convierte las variables categóricas (`Genero`, `Nivel_Educativo`, `Ciudad`) en variables binarias usando codificación *one-hot*.  
  La opción `drop_first=True` elimina una categoría por cada variable para evitar colinealidad.

---

#### 📏 Escalado de variables numéricas

- `MinMaxScaler()`:  
  Escala los valores de `Edad` e `Ingreso` entre 0 y 1, lo cual es útil para algoritmos sensibles a la magnitud de los datos (como KNN o regresión logística).

- `scaler.fit_transform(...)`:  
  Ajusta el escalador a los datos y transforma simultáneamente los valores.

---

El resultado final es un DataFrame `df_clean` sin valores nulos, con variables categóricas codificadas y variables numéricas escaladas, listo para usar en un modelo predictivo.


## 4. Evaluación del modelo después de la limpieza

In [4]:
Xc = df_clean.drop(columns=['ID', 'CodigoID', 'Horas_Internet', 'Alta_conectividad'])
yc = df_clean['Alta_conectividad']

Xc_train, Xc_test, yc_train, yc_test = train_test_split(Xc, yc, test_size=0.3, random_state=42)

model_after = LogisticRegression(max_iter=200)
model_after.fit(Xc_train, yc_train)
yc_pred = model_after.predict(Xc_test)
acc_after = accuracy_score(yc_test, yc_pred)

print("✅ Exactitud después de la limpieza:", round(acc_after, 4))

✅ Exactitud después de la limpieza: 0.6205


### 🤖 Modelado con datos limpios – Regresión logística

- `Xc = df_clean.drop(columns=['ID', 'CodigoID', 'Horas_Internet', 'Alta_conectividad'])`:  
  Se seleccionan como variables predictoras (`Xc`) todas las columnas del dataset limpio excepto identificadores, la variable a predecir (`Alta_conectividad`) y la columna `Horas_Internet` (que se usó para crear la variable objetivo).

- `yc = df_clean['Alta_conectividad']`:  
  Define la variable objetivo (`yc`), es decir, lo que se quiere predecir: si una persona tiene o no alta conectividad.

---

#### 📦 División del dataset

- `train_test_split(...)`:  
  Se divide el conjunto de datos en entrenamiento (70%) y prueba (30%) usando una semilla aleatoria fija (`random_state=42`) para garantizar resultados reproducibles.

---

#### 🔍 Entrenamiento del modelo

- `LogisticRegression(max_iter=200)`:  
  Se crea un modelo de regresión logística para clasificación binaria. El parámetro `max_iter=200` define el número máximo de iteraciones para ajustar el modelo si los datos son complejos.

- `model_after.fit(Xc_train, yc_train)`:  
  Se entrena el modelo con los datos de entrenamiento.

---

#### 🧠 Predicción y evaluación

- `yc_pred = model_after.predict(Xc_test)`:  
  El modelo genera predicciones para el conjunto de prueba.

- `accuracy_score(yc_test, yc_pred)`:  
  Se calcula la **exactitud** del modelo: la proporción de predicciones correctas sobre el total.

- `print(...)`:  
  Imprime en pantalla la exactitud obtenida redondeada a 4 cifras decimales.

---

Este bloque permite evaluar el impacto de la limpieza sobre el rendimiento del modelo. Una mayor exactitud respecto al modelo sin limpiar indica que la preparación de datos fue beneficiosa.


### ¿Qué significa este resultado?

`Exactitud después de la limpieza: 0.6205`

Este valor indica que el modelo de regresión logística logró una **exactitud del 62.05%** al predecir si una persona tiene o no alta conectividad, utilizando un conjunto de datos que fue previamente **preparado y limpiado**.

---

### ¿Qué se hizo antes de este resultado?

Antes de entrenar este modelo, se aplicaron varias técnicas de **preparación de datos**:

- **Imputación de valores faltantes**:  
  Se reemplazaron los valores nulos en `Edad` con la mediana y en `Ingreso` con la media.

- **Codificación de variables categóricas**:  
  Se transformaron las variables como `Género`, `Nivel_Educativo` y `Ciudad` en variables numéricas usando `one-hot encoding`.

- **Escalado de variables numéricas**:  
  `Edad` e `Ingreso` fueron normalizadas con **Min-Max Scaling** para que estén en la misma escala (entre 0 y 1).

---

### ¿Por qué mejoró la exactitud?

- El modelo ahora tiene acceso a **más información útil**, no solo `Edad` e `Ingreso`.
- Se retuvieron registros con datos faltantes gracias a la **imputación**, evitando la pérdida de datos.
- Las variables categóricas están representadas numéricamente, lo que mejora la capacidad del modelo para identificar patrones.
- El escalado garantiza que ninguna variable domine por su magnitud.

---

### ¿Cómo interpretar este valor?

- Una exactitud del **62.05%** es **superior al modelo anterior** (57.78%), aunque no representa una mejora extrema.
- La diferencia muestra que preparar los datos **tiene un impacto positivo**, pero también que el modelo puede beneficiarse aún más con técnicas adicionales o mejores variables.

---

### ¿Por qué es importante esta comparación?

Comparar este resultado con el obtenido antes de la limpieza permite **cuantificar el impacto real de preparar bien los datos**.  
Aunque la mejora no es drástica, demuestra que **incluso mejoras modestas pueden hacer que el modelo sea más confiable**.

---

### Conclusión

El modelo después de la limpieza supera en precisión al modelo inicial.  
Esto valida que **limpiar, imputar, codificar y escalar adecuadamente los datos mejora el rendimiento predictivo**, y es una parte crítica del proceso de análisis predictivo.

La comparación entre ambos enfoques demuestra cómo una limpieza adecuada mejora la capacidad del modelo para generalizar.

Antes de la limpieza: solo dos variables, filas eliminadas por nulos.
Después de la limpieza: más variables, nulos imputados, todo escalado
Esta diferencia se refleja directamente en la exactitud del modelo.

👉 Las decisiones de preparación de datos son tan importantes como el modelo mismo.
